In [6]:
import pandas as pd
import pyarrow.parquet as pq
import gzip

In [2]:
steam1 = pd.read_csv('./Data\Data-Limpia\steam_games_limpio.csv')

items = pd.read_csv('./Data/Data-Limpia/user_items_limpio.csv')

steam1 = steam1[['genres','id','año']]
items = items[['user_id','item_id','playtime_forever']]

In [3]:
steam1 = steam1.groupby('id').first().reset_index()

steam1 = steam1[steam1['año']!=0]

resultado = steam1.merge(items, left_on='id', right_on='item_id', how='inner')


In [4]:
resultado.head()

,id,genres,año,user_id,item_id,playtime_forever
0,10.0,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",2000,76561197970982479,10.0,0.10
1,10.0,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",2000,js41637,10.0,0.00
2,10.0,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",2000,Riot-Punch,10.0,0.00
3,10.0,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",2000,doctr,10.0,1.55
4,10.0,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",2000,corrupted_soul,10.0,1.80


In [5]:
resultado['playtime_forever'] = resultado['playtime_forever'].astype(int)

In [7]:
with gzip.open('./Data/Data-Funciones/F_user_genre.csv.gz', 'wb') as f:
    resultado.to_csv(f, index=False, encoding='utf-8')

In [7]:
resultado.to_parquet('df-userforgenre.parquet')

In [8]:
def UserForGenre(genero):
    #df = pd.read_parquet('df-userforgenre.parquet')
    df = pd.read_csv('./Data/Data-Funciones/F_user_genre.csv.gz', compression='gzip')

    df_genero= df.groupby(['user_id', 'año']).sum().reset_index()

    df_genero = df_genero[df_genero['genres'].str.contains(genero)]

    # Encontrar al usuario con la máxima cantidad de playtime
    usuario_max_playtime = df_genero.loc[df_genero['playtime_forever'].idxmax()]['user_id']

    # Filtrar el DataFrame original por usuario y género
    df_usuario_genero = df[(df['user_id'] == usuario_max_playtime)]
    # Agrupar por año y sumar el tiempo de juego
    poranio = df_usuario_genero.groupby('año')['playtime_forever'].sum().to_dict()

    # Crear un diccionario con la información
    dicc = {
        'usuario': usuario_max_playtime,
        'años': poranio
    }

    return dicc

In [9]:
UserForGenre('Action')

{'usuario': 'REBAS_AS_F-T',
 'años': {1996: 0,
  1998: 0,
  1999: 3,
  2000: 0,
  2001: 0,
  2002: 0,
  2003: 31,
  2004: 150,
  2005: 54,
  2006: 28,
  2007: 169,
  2008: 81,
  2009: 1014,
  2010: 1199,
  2011: 3104,
  2012: 4421,
  2013: 6741,
  2014: 7951,
  2015: 17810,
  2016: 22117,
  2017: 579}}